In [45]:
import pandas as pd

In [51]:
df=pd.read_csv('https://raw.githubusercontent.com/ClarksonAirlab/2019_sensor_workshop/master/Python/Tutorial/Tutorial%20Data/cornwall.csv')

In [52]:
df=df.dropna()

In [55]:
pd.to_datetime(dict(year=df.year, month=df.month, day=1))

0      2018-07-01
1      2018-07-01
2      2018-07-01
3      2018-07-01
4      2018-07-01
5      2018-07-01
6      2018-07-01
7      2018-07-01
8      2018-07-01
9      2018-07-01
10     2018-07-01
11     2018-07-01
12     2018-07-01
13     2018-07-01
14     2018-07-01
15     2018-07-01
16     2018-07-01
17     2018-07-01
18     2018-07-01
19     2018-07-01
20     2018-07-01
21     2018-07-01
22     2018-07-01
23     2018-07-01
24     2018-07-01
25     2018-07-01
26     2018-07-01
27     2018-07-01
28     2018-07-01
29     2018-07-01
          ...    
9580   2019-07-01
9581   2019-07-01
9582   2019-07-01
9583   2019-07-01
9584   2019-07-01
9585   2019-07-01
9586   2019-07-01
9587   2019-07-01
9588   2019-07-01
9589   2019-07-01
9590   2019-07-01
9591   2019-07-01
9592   2019-07-01
9593   2019-07-01
9594   2019-07-01
9595   2019-07-01
9596   2019-07-01
9597   2019-07-01
9598   2019-07-01
9599   2019-07-01
9600   2019-07-01
9601   2019-07-01
9602   2019-07-01
9603   2019-07-01
9604   201

In [49]:
df = df.dropna(how='any',axis=0)

In [50]:
df

,year,month,date,hour,o3,pm2.5,No2
0,2018,7,1,0,52,21,2.6
1,2018,7,1,1,50,20,1.6
2,2018,7,1,2,46,19,1.7
3,2018,7,1,3,42,19,1.6
4,2018,7,1,4,39,20,1.7
5,2018,7,1,5,37,19,2.8
6,2018,7,1,6,N/A,19,N/A
7,2018,7,1,7,37,18,1.4
8,2018,7,1,8,37,18,1.7
9,2018,7,1,9,36,17,1.5
